In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
#PATH = '/content/drive/My Drive/PPM_Stability/'
#PATH = "C:/Users/velmurug/Documents/Stability Experiments/benchmark_interpretability/PPM_Stability/"
#PATH = "C:/Users/Mythreyi/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
#PATH = "C:/Users/mythr/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
PATH = "C:/Users/n9455647/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
sys.path.append(PATH)

In [2]:
from DatasetManager import DatasetManager

import pandas as pd
import numpy as np

import os
import sys
from sys import argv
import pickle
import random
import joblib

import statistics
import math

import warnings
warnings.filterwarnings('ignore')

In [37]:
datasets = ["production", "sepsis_cases", "bpic2012"]
methods = ["xgboost_single_agg", "xgboost_prefix_agg", "xgboost_prefix_index", "lstm_single_3d"]
dataset_names = {"production":"production", "sepsis_cases":"sepsis_cases_1", "bpic2012":"bpic2012_accepted"}


for data in datasets:
    print(data)
            
    dataset_manager = DatasetManager(dataset_names[data])

    
    for method in methods:
        print(method)
        folder_loc = os.path.join(PATH, "%s/%s/" %(data, method))
        
        if "single" in method:
            num_buckets = 1
        else:
            num_buckets = len([name for name in os.listdir(os.path.join(folder_loc,'models/'))])
            
        for bucket in range(num_buckets):
            if "lstm" in method:
                bucketID = "all"
            else:
                bucketID = bucket+1
            print ('Bucket', bucketID)
            
            X_train_path = os.path.join(folder_loc, "train_data/bucket_%s_prefixes.pickle" % (bucketID))
            with open (X_train_path, 'rb') as f:
                dt_train_bucket = pickle.load(f)

            X_test_path = os.path.join(folder_loc, "test_data/bucket_%s_prefixes.pickle" % (bucketID))
            with open (X_test_path, 'rb') as f:
                dt_test_bucket = pickle.load(f)
            

            if "lstm" not in method: 
                y_train = dataset_manager.get_label_numeric(dt_train_bucket)
                y_test = dataset_manager.get_label_numeric(dt_test_bucket)
            
                train_pos_cases = len([i for i in y_train if i == 1])
                train_neg_cases = len([i for i in y_train if i == 0])
                train_pos_ratio = train_pos_cases/(train_pos_cases + train_neg_cases)
                print("\tPositives in training set:", train_pos_ratio)

                test_pos_cases = len([i for i in y_test if i == 1])
                test_neg_cases = len([i for i in y_test if i == 0])
                test_pos_ratio = test_pos_cases/(test_pos_cases + test_neg_cases)
                print("\tPositives in testing set:", test_pos_ratio)
                
                feat_comb_path = os.path.join(folder_loc, "bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (bucketID))
                with open (feat_comb_path, 'rb') as f:
                    feature_combiner = joblib.load(f)
                
                test_x = feature_combiner.transform(dt_test_bucket)
                vector_length = test_x[0].shape
                
                print("Feature Vector Length:", vector_length)
                
            else:
                y_train_path = os.path.join(folder_loc, "train_data/bucket_%s_labels.pickle" % (bucketID))
                with open (y_train_path, 'rb') as f:
                    train_y = pickle.load(f)

                y_test_path = os.path.join(folder_loc, "test_data/bucket_%s_labels.pickle" % (bucketID))
                with open (y_test_path, 'rb') as f:
                    test_y = pickle.load(f)
                
                y_train = [int(np.where(i==1)[0]) for i in train_y]
                y_test = test_y
               
                train_pos_cases = len([i for i in y_train if i == 1])
                train_neg_cases = len([i for i in y_train if i == 0])
                train_pos_ratio = train_pos_cases/(train_pos_cases + train_neg_cases)
                print("\tPositives in training set:", train_pos_ratio)

                test_pos_cases = len([i for i in y_test if i == 1])
                test_neg_cases = len([i for i in y_test if i == 0])
                test_pos_ratio = test_pos_cases/(test_pos_cases + test_neg_cases)
                print("\tPositives in testing set:", test_pos_ratio)
                
                vector_length = dt_test_bucket[0].shape
                
                print("Feature Vector Length:", vector_length)

production
xgboost_single_agg
Bucket 1
	Positives in training set: 0.5879360465116279
	Positives in testing set: 0.4598155467720685
Feature Vector Length: (162,)
xgboost_prefix_agg
Bucket 1
	Positives in training set: 0.5984848484848485
	Positives in testing set: 0.4318181818181818
Feature Vector Length: (146,)
Bucket 2
	Positives in training set: 0.6031746031746031
	Positives in testing set: 0.4523809523809524
Feature Vector Length: (153,)
Bucket 3
	Positives in training set: 0.6083333333333333
	Positives in testing set: 0.46153846153846156
Feature Vector Length: (150,)
Bucket 4
	Positives in training set: 0.5909090909090909
	Positives in testing set: 0.4411764705882353
Feature Vector Length: (150,)
Bucket 5
	Positives in training set: 0.5555555555555556
	Positives in testing set: 0.4576271186440678
Feature Vector Length: (150,)
Bucket 6
	Positives in training set: 0.550561797752809
	Positives in testing set: 0.4444444444444444
Feature Vector Length: (156,)
Bucket 7
	Positives in trai

Feature Vector Length: (201,)
Bucket 16
	Positives in training set: 0.19736842105263158
	Positives in testing set: 0.1
Feature Vector Length: (195,)
Bucket 17
	Positives in training set: 0.18333333333333332
	Positives in testing set: 0.10204081632653061
Feature Vector Length: (184,)
Bucket 18
	Positives in training set: 0.16447368421052633
	Positives in testing set: 0.05263157894736842
Feature Vector Length: (180,)
Bucket 19
	Positives in training set: 0.16541353383458646
	Positives in testing set: 0.05555555555555555
Feature Vector Length: (172,)
Bucket 20
	Positives in training set: 0.18181818181818182
	Positives in testing set: 0.0625
Feature Vector Length: (176,)
Bucket 21
	Positives in training set: 0.18181818181818182
	Positives in testing set: 0.07142857142857142
Feature Vector Length: (170,)
Bucket 22
	Positives in training set: 0.16470588235294117
	Positives in testing set: 0.08333333333333333
Feature Vector Length: (159,)
Bucket 23
	Positives in training set: 0.17105263157894

Feature Vector Length: (130,)
Bucket 19
	Positives in training set: 0.4861351819757366
	Positives in testing set: 0.5373467112597548
Feature Vector Length: (131,)
Bucket 20
	Positives in training set: 0.4896168470312957
	Positives in testing set: 0.5385474860335195
Feature Vector Length: (133,)
Bucket 21
	Positives in training set: 0.5047155460906602
	Positives in testing set: 0.5527522935779816
Feature Vector Length: (134,)
Bucket 22
	Positives in training set: 0.5088262620006194
	Positives in testing set: 0.5578703703703703
Feature Vector Length: (134,)
Bucket 23
	Positives in training set: 0.5148063781321185
	Positives in testing set: 0.5686746987951807
Feature Vector Length: (134,)
Bucket 24
	Positives in training set: 0.5203709837694601
	Positives in testing set: 0.5798525798525799
Feature Vector Length: (134,)
Bucket 25
	Positives in training set: 0.523709167544784
	Positives in testing set: 0.577922077922078
Feature Vector Length: (134,)
Bucket 26
	Positives in training set: 0.5

Feature Vector Length: (2829,)
lstm_single_3d
Bucket all
	Positives in training set: 0.49785241817713255
	Positives in testing set: 0.5539380487645658
Feature Vector Length: (40, 102)
